# PVGIS Data Analysis - Teste e Validação

Notebook para testar e validar a obtenção e processamento de dados PVGIS.
Baseado no código do pvlib-service.

In [ ]:
# Instalação das dependências necessárias
!pip install pandas numpy requests matplotlib seaborn jupyter ipython
!pip install pvlib-python scipy scikit-learn

# Verificar instalação
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Todas as dependências foram instaladas com sucesso!")
print(f"📊 Pandas versão: {pd.__version__}")
print(f"🔢 NumPy versão: {np.__version__}")
print(f"🌐 Requests disponível")
print(f"📈 Matplotlib disponível")
print(f"🎨 Seaborn disponível")

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Configurações de plotagem
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 10

## 📍 Dados de Entrada
Defina aqui os parâmetros para teste:

In [3]:
# PARÂMETROS DE ENTRADA - EDITE AQUI
LATITUDE = -23.5505  # São Paulo
LONGITUDE = -46.6333
TILT = 23  # Inclinação dos painéis (graus)
AZIMUTH = 0  # Azimute (0 = Norte)
START_YEAR = 2005
END_YEAR = 2020
PVGIS_TIMEOUT = 120

print(f"📍 Localização: {LATITUDE}, {LONGITUDE}")
print(f"🔧 Configuração: Tilt={TILT}°, Azimute={AZIMUTH}°")
print(f"📅 Período: {START_YEAR}-{END_YEAR}")

📍 Localização: -23.5505, -46.6333
🔧 Configuração: Tilt=23°, Azimute=0°
📅 Período: 2005-2020


## 🛠️ Funções PVGIS (extraídas do pvlib-service)

In [ ]:
class PVGISService:
    """Serviço para buscar dados PVGIS - Baseado no pvlib-service"""
    
    def __init__(self, base_url: str = "https://re.jrc.ec.europa.eu/api/v5_2"):
        self.base_url = base_url
    
    def fetch_hourly_data(self, lat: float, lon: float, start_year: int = 2005, 
                         end_year: int = 2020, tilt: float = 0, azimuth: float = 0, 
                         timeout: int = 120) -> Dict:
        """Buscar dados horários do PVGIS"""
        
        url = f"{self.base_url}/seriescalc"
        params = {
            'lat': lat,
            'lon': lon,
            'startyear': start_year,
            'endyear': end_year,
            'outputformat': 'json',
            'usehorizon': 1,
            'selectrad': 1,
            'angle': tilt,
            'aspect': azimuth
        }
        
        print(f"🔄 Fazendo requisição PVGIS: {url}")
        print(f"📊 Parâmetros: {params}")
        
        try:
            response = requests.get(url, params=params, timeout=timeout)
            response.raise_for_status()
            
            data = response.json()
            print(f"✅ Dados recebidos: {len(data.get('outputs', {}).get('hourly', []))} registros")
            
            return data
            
        except Exception as e:
            print(f"❌ Erro ao buscar dados PVGIS: {e}")
            raise
    
    def process_hourly_data(self, raw_data: Dict) -> pd.DataFrame:
        """Processar dados horários em DataFrame"""
        
        if 'outputs' not in raw_data or 'hourly' not in raw_data['outputs']:
            raise ValueError("Dados PVGIS inválidos")
        
        hourly_data = raw_data['outputs']['hourly']
        print(f"🔄 Processando {len(hourly_data)} registros horários...")
        
        # Converter para DataFrame
        df = pd.DataFrame(hourly_data)
        
        # Criar coluna de timestamp
        df['timestamp'] = pd.to_datetime(
            df['time'], format='%Y%m%d:%H%M', errors='coerce'
        )
        
        # Adicionar colunas de data
        df['year'] = df['timestamp'].dt.year
        df['month'] = df['timestamp'].dt.month
        df['day'] = df['timestamp'].dt.day
        df['hour'] = df['timestamp'].dt.hour
        
        # Converter colunas numéricas
        numeric_cols = ['G(i)', 'H_sun', 'T2m', 'WS10m', 'Int']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Filtrar dados inválidos
        df = df.dropna(subset=['timestamp', 'G(i)'])
        
        print(f"✅ DataFrame criado: {len(df)} registros válidos")
        print(f"📅 Período: {df['timestamp'].min()} a {df['timestamp'].max()}")
        
        return df
    
    def calculate_monthly_averages(self, df: pd.DataFrame) -> Dict:
        """Calcular médias mensais"""
        
        # Agrupar por mês e calcular médias
        monthly = df.groupby('month').agg({
            'G(i)': 'mean',
            'H_sun': 'mean', 
            'T2m': 'mean',
            'WS10m': 'mean'
        }).round(2)
        
        # Converter para listas (formato da API)
        result = {
            'irradiacao_mensal': monthly['G(i)'].tolist(),
            'temperatura_media': monthly['T2m'].tolist(),
            'vento_medio': monthly['WS10m'].tolist(),
            'horas_sol': monthly['H_sun'].tolist()
        }
        
        # Estatísticas gerais
        result['media_anual'] = monthly['G(i)'].mean()
        result['maximo'] = monthly['G(i)'].max()
        result['minimo'] = monthly['G(i)'].min()
        result['variacao_sazonal'] = result['maximo'] - result['minimo']
        
        print(f"📊 Irradiação média anual: {result['media_anual']:.2f} kWh/m²/dia")
        print(f"📈 Variação sazonal: {result['variacao_sazonal']:.2f} kWh/m²/dia")
        
        return result

## 🌐 Buscar Dados PVGIS

In [7]:
# Inicializar serviço PVGIS
pvgis = PVGISService()

# Buscar dados
print("🚀 Iniciando busca de dados PVGIS...")
raw_data = pvgis.fetch_hourly_data(
    lat=LATITUDE,
    lon=LONGITUDE,
    start_year=START_YEAR,
    end_year=END_YEAR,
    tilt=TILT,
    azimuth=AZIMUTH,
    timeout=PVGIS_TIMEOUT
)

print("\n📋 Estrutura dos dados recebidos:")
print(f"- Chaves principais: {list(raw_data.keys())}")
if 'outputs' in raw_data:
    print(f"- Outputs: {list(raw_data['outputs'].keys())}")
if 'inputs' in raw_data:
    print(f"- Inputs: {raw_data['inputs']}")

🚀 Iniciando busca de dados PVGIS...
🔄 Fazendo requisição PVGIS: https://re.jrc.ec.europa.eu/api/v5_2/tmy
📊 Parâmetros: {'lat': -23.5505, 'lon': -46.6333, 'startyear': 2005, 'endyear': 2020, 'outputformat': 'json', 'usehorizon': 1, 'selectrad': 1, 'angle': 23, 'aspect': 0}
✅ Dados recebidos: 0 registros

📋 Estrutura dos dados recebidos:
- Chaves principais: ['inputs', 'outputs', 'meta']
- Outputs: ['months_selected', 'tmy_hourly']
- Inputs: {'location': {'latitude': -23.5505, 'longitude': -46.6333, 'elevation': 778.0, 'irradiance_time_offset': 0.0577}, 'meteo_data': {'radiation_db': 'PVGIS-SARAH2', 'meteo_db': 'ERA5', 'year_min': 2005, 'year_max': 2020, 'use_horizon': True, 'horizon_db': 'DEM-calculated'}}


In [ ]:
# Processar dados
print("🔄 Processando dados horários...")
df_hourly = pvgis.process_hourly_data(raw_data)

# Mostrar primeiras linhas
print("\n📊 Primeiras linhas dos dados processados:")
display(df_hourly.head(10))

print("\n📈 Estatísticas básicas:")
display(df_hourly[['G(i)', 'T2m', 'WS10m', 'H_sun']].describe())

In [ ]:
# Calcular médias mensais
print("🗓️ Calculando médias mensais...")
monthly_data = pvgis.calculate_monthly_averages(df_hourly)

print("\n📊 Dados mensais:")
meses = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for i, mes in enumerate(meses):
    print(f"{mes}: {monthly_data['irradiacao_mensal'][i]:.2f} kWh/m²/dia")

## 📊 Visualizações dos Dados PVGIS

In [ ]:
# Gráfico de irradiação mensal
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Irradiação mensal
ax1.bar(meses, monthly_data['irradiacao_mensal'], color='orange', alpha=0.7)
ax1.set_title('Irradiação Solar Mensal (PVGIS)', fontsize=14, fontweight='bold')
ax1.set_ylabel('kWh/m²/dia')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Temperatura média
ax2.plot(meses, monthly_data['temperatura_media'], marker='o', color='red', linewidth=2)
ax2.set_title('Temperatura Média Mensal', fontsize=14, fontweight='bold')
ax2.set_ylabel('°C')
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# Vento médio
ax3.bar(meses, monthly_data['vento_medio'], color='lightblue', alpha=0.7)
ax3.set_title('Velocidade do Vento Média', fontsize=14, fontweight='bold')
ax3.set_ylabel('m/s')
ax3.grid(True, alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

# Horas de sol
ax4.plot(meses, monthly_data['horas_sol'], marker='s', color='gold', linewidth=2)
ax4.set_title('Horas de Sol Mensais', fontsize=14, fontweight='bold')
ax4.set_ylabel('horas')
ax4.grid(True, alpha=0.3)
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"📊 Resumo dos Dados PVGIS:")
print(f"🔸 Irradiação média anual: {monthly_data['media_anual']:.2f} kWh/m²/dia")
print(f"🔸 Mês com maior irradiação: {meses[monthly_data['irradiacao_mensal'].index(monthly_data['maximo'])]} ({monthly_data['maximo']:.2f})")
print(f"🔸 Mês com menor irradiação: {meses[monthly_data['irradiacao_mensal'].index(monthly_data['minimo'])]} ({monthly_data['minimo']:.2f})")
print(f"🔸 Variação sazonal: {monthly_data['variacao_sazonal']:.2f} kWh/m²/dia")

## 💾 Salvar Dados para Próximos Notebooks

In [ ]:
# Salvar dados processados
import os

# Criar diretório se não existir
os.makedirs('../data', exist_ok=True)

# Salvar DataFrame completo
df_hourly.to_csv('../data/pvgis_hourly_data.csv', index=False)

# Salvar dados mensais
with open('../data/pvgis_monthly_data.json', 'w') as f:
    json.dump(monthly_data, f, indent=2)

# Salvar configuração usada
config = {
    'latitude': LATITUDE,
    'longitude': LONGITUDE,
    'tilt': TILT,
    'azimuth': AZIMUTH,
    'start_year': START_YEAR,
    'end_year': END_YEAR
}

with open('../data/config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("💾 Dados salvos em:")
print("- ../data/pvgis_hourly_data.csv")
print("- ../data/pvgis_monthly_data.json")
print("- ../data/config.json")
print("\n✅ Dados PVGIS prontos para uso nos próximos notebooks!")